# **Project Name**    -Credit Score Classification for Paisabazaar



# **Project Summary -**

Paisabazaar, a financial services platform, helps customers access loans and credit products. Credit score is a key metric for lenders to assess repayment capacity and manage risks. This project focuses on predicting credit scores (Good, Standard, Poor) using customer financial and behavioral data.

## Data Collection and Cleaning :

The dataset of 100,000 records included income, debt, loan types, repayment history, and demographics. Raw data was cleaned by handling missing values, encoding categorical variables (e.g., occupation, payment behavior), and normalizing numerical features like income and outstanding debt.
A special challenge was the Type of Loan column, where customers often had multiple loans (auto, personal, home equity, etc.). This was managed by creating binary indicators for each loan type. The processed dataset provided a reliable foundation for analysis.

## Data Visualization and Insights :

Exploratory Data Analysis revealed that:

* Income & Balance: Higher income and steady balances were linked to good credit scores.

* Repayment Behavior: Frequent delays and high outstanding debt strongly correlated with poor scores.

* Loan Types: Customers juggling multiple loans, especially personal and auto loans, faced higher risks.

* Credit Utilization: High utilization ratios indicated financial stress and poor creditworthiness.

Overall, behavioral consistency in payments emerged as more important than income alone.

## Model Development and Results:

Machine learning models were built using Random Forest and XGBoost. Data was split into 80% training and 20% testing. XGBoost achieved the best performance, with strong accuracy and recall. Feature importance analysis confirmed that delayed payments, outstanding debt, annual income, and utilization ratio were the strongest predictors of credit score.



The project showed that credit scores can be reliably predicted through financial and behavioral data.By integrating this model, Paisabazaar can:

* Flag high-risk customers early.

* Offer tailored products to different credit score segments.

* Improve risk management while providing fairer, personalized financial solutions.

This approach strengthens decision-making and enhances both customer trust and business outcomes.



# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


>1.Paisabazaar needs to accurately assess the creditworthiness of its customers to make informed loan approval and risk management decisions.

>2.Current processes require a systematic method to classify individuals’ credit scores using available customer data.

>3.Predicting credit scores based on features like income, credit card usage, and payment behavior will help reduce loan default risks.

>4.Accurate credit score classification can enable personalized financial product recommendations and improve overall customer service.

>5.The goal is to develop a predictive model that can classify individuals’ credit scores effectively, supporting better financial decision-making.

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import joblib
import os
RND = 42

### Dataset Loading

In [ ]:
# Load Dataset
df = pd.read_csv("/content/drive/MyDrive/dataset-2.csv")

### Dataset First View

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Dataset First Look
display(df.head())

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
print("Shape:", df.shape)

### Dataset Information

In [ ]:
# Dataset Info
df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print("Duplicate Values:", df.duplicated().sum())

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
df.isnull().sum()

### What did you know about your dataset?

# Dataset Description: Credit Score Prediction

The dataset consists of 100,000 records with 27 features related to customer financial behavior and demographic details. The goal is to predict the Credit_Score of individuals.

## 1. Customer and Demographic Information

* ID / Customer_ID (int64): Unique identifiers for each record and customer.

* Name (object): Customer name.

* Age (float64): Age of the customer.

* SSN (float64): Social Security Number of the customer.

* Occupation (object): Profession of the customer.

## 2. Income and Salary Details

* Annual_Income (float64): Yearly income of the customer.

* Monthly_Inhand_Salary (float64): Monthly take-home salary after deductions.

## 3. Banking and Credit Details

* Interest_Rate (float64): Interest rate applicable on loans or credit.

* Num_of_Loan (float64): Number of loans taken.

* Type_of_Loan (object): Type/category of loans held.

* Delay_from_due_date (float64): Number of days payment was delayed from due date.

* Num_of_Delayed_Payment (float64): Count of delayed payments.

* Changed_Credit_Limit (float64): Number of times the credit limit was changed.

* Num_Credit_Inquiries (float64): Number of credit inquiries in the recent period.

* Credit_Mix (object): Mix of different credit types (good, average, poor).

* Outstanding_Debt (float64): Total outstanding debt.

* Credit_Utilization_Ratio (float64): Ratio of used credit to available credit.

* Credit_History_Age (float64): Duration of credit history in months or years.

* Payment_of_Min_Amount (object): Indicates if the minimum payment is paid regularly.

* Total_EMI_per_month (float64): Total monthly EMI for loans.

* Amount_invested_monthly (float64): Monthly investments made by the customer.

* Payment_Behaviour (object): Customer payment behavior (e.g., regular, delayed).

* Monthly_Balance (float64): Monthly balance in bank accounts.

## 4. Target Variable

* Credit_Score (object): Classification of credit score (e.g., Poor, Standard, Good).

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
df.columns

In [ ]:
# Dataset Describe
df.describe()

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
df.nunique()

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.

def process_loan_count(df):
    """
    Function to clean and create a final loan count column
    using both Num_of_Loan and Type_of_Loan.
    """

    # 1. Ensure Num_of_Loan is numeric
    df["Num_of_Loan"] = pd.to_numeric(df["Num_of_Loan"], errors="coerce")

    # 2. Extract loan count from Type_of_Loan
    # - If string: count commas
    # - If missing/NaN: assign 0
    df["Loan_Count_From_Type"] = df["Type_of_Loan"].apply(
        lambda x: len(x.split(",")) if isinstance(x, str) else 0
    )

    # 3. Create Final Loan Count column
    df["Final_Loan_Count"] = df["Num_of_Loan"].fillna(df["Loan_Count_From_Type"]).astype(int)

    return df

df = process_loan_count(df)

print(df[["Num_of_Loan", "Type_of_Loan", "Loan_Count_From_Type", "Final_Loan_Count"]].head())


In [ ]:
# Drop obviously irrelevant or sensitive columns if present
for col in ["ID", "Name", "SSN"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

## 4. Data Vizualization

## Univarient Analysis

#### Distributions of Age

In [ ]:
# Numerical distributions
sns.histplot(df['Age'],bins=5,color='olive',kde=True)
plt.suptitle(" Distributions of Age")
plt.show()

Most of the Customers are middle aged.

####  Distributions of Annual_Income

In [ ]:
# Numerical distributions
sns.histplot(df['Annual_Income'],bins=10,color='olive',kde=True)
plt.suptitle(" Distributions of Annual_Income")
plt.show()

##### chat shows that maximum number of people's annual income is less than 50000

#### Distributions of Num_of_Loan

In [ ]:
# Numerical distributions
sns.histplot(df['Num_of_Loan'],bins=9,color='olive',kde=True)
plt.suptitle(" Distributions of Num_of_Loan")
plt.show()

#### Distributions of Num_of_Delayed_Payment

In [ ]:
# Numerical distributions
sns.histplot(df['Num_of_Delayed_Payment'],bins=5,color='olive',kde=True)
plt.suptitle(" Distributions of Num_of_Delayed_Payment")
plt.show()

#### Distributions of Outstanding_Debt

In [ ]:
# Numerical distributions
sns.histplot(df['Outstanding_Debt'],bins=10,color='olive',kde=True)
plt.suptitle(" Distributions of Outstanding_Debt")
plt.show()

#### Distribution of Occupation

In [ ]:
# Categorical counts
plt.figure(figsize=(6,4))
sns.countplot(data=df, x='Occupation', order=df['Occupation'].value_counts().index)
plt.title(f"Distribution of {'Occupation'}")
plt.xticks(rotation=45)
plt.show()

#### Distribution of Credit_Mix

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df, x='Credit_Mix', order=df['Credit_Mix'].value_counts().index)
plt.title(f"Distribution of {'Credit_Mix'}")
plt.xticks(rotation=45)
plt.show()

#### Distribution of Payment_Behaviour

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df, x='Payment_Behaviour', order=df['Payment_Behaviour'].value_counts().index)
plt.title(f"Distribution of {'Payment_Behaviour'}")
plt.xticks(rotation=45)
plt.show()

#### Distribution of Credit_Score

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df, x='Credit_Score', order=df['Credit_Score'].value_counts().index)
plt.title(f"Distribution of {'Credit_Score'}")
plt.xticks(rotation=45)
plt.show()

#### Box plot of Credit Score

In [ ]:
sns.boxplot(x='Credit_Score',data=df)
plt.title("Box plot of Credit Score")
plt.show()

# Bivariente Analysis

#### Annual_Income vs Credit Score

In [ ]:
# Credit Score vs Numeric features
plt.figure(figsize=(6,4))
sns.boxplot(x="Credit_Score", y="Annual_Income", data=df, palette="Set2")
plt.title(f"{"Annual_Income"} vs Credit Score")
plt.show()

#### Outstanding_Debt vs Credit Score

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x="Credit_Score", y="Outstanding_Debt", data=df, palette="Set2")
plt.title(f"{"Outstanding_Debt"} vs Credit Score")
plt.show()

#### Credit_Utilization_Ratio vs Credit Score

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x="Credit_Score", y="Credit_Utilization_Ratio", data=df, palette="Set2")
plt.title(f"{"Credit_Utilization_Ratio"} vs Credit Score")
plt.show()

#### Monthly_Balance vs Credit Score

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x="Credit_Score", y="Monthly_Balance", data=df, palette="Set2")
plt.title(f"{"Monthly_Balance"} vs Credit Score")
plt.show()

#### Num_of_Delayed_Payment vs Credit Score

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x="Credit_Score", y="Num_of_Delayed_Payment", data=df, palette="Set2")
plt.title(f"{"Num_of_Delayed_Payment"} vs Credit Score")
plt.show()

#### Occupation vs Credit Score

In [ ]:
# Credit Score vs Categorical
plt.figure(figsize=(10,8))
sns.countplot(x="Occupation", hue="Credit_Score",legend=False, data=df)
plt.title(f"{"Occupation"} vs Credit Score")
plt.xticks(rotation=45)
plt.show()

#### Credit_Mix vs Credit Score

In [ ]:
# Credit Score vs Categorical
plt.figure(figsize=(6,4))
sns.countplot(x="Credit_Mix", hue="Credit_Score", data=df)
plt.title(f"{"Credit_Mix"} vs Credit Score")
plt.xticks(rotation=45)
plt.show()

#### Payment_of_Min_Amount vs Credit Score

In [ ]:
# Credit Score vs Categorical
plt.figure(figsize=(6,4))
sns.countplot(x="Payment_of_Min_Amount", hue="Credit_Score", data=df)
plt.title(f"{"Payment_of_Min_Amount"} vs Credit Score")
plt.xticks(rotation=45)
plt.show()

#### Payment_Behaviour vs Credit Score

In [ ]:
# Credit Score vs Categorical
plt.figure(figsize=(16,4))
sns.countplot(x="Payment_Behaviour", hue="Credit_Score", data=df)
plt.title(f"{"Payment_Behaviour"} vs Credit Score")
plt.xticks(rotation=45)
plt.show()

# Multivariate Analysis

#### Correlation Heatmap

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12,8))
corr = df.corr(numeric_only=True)
sns.heatmap(corr, cmap="coolwarm", annot=False)
plt.title("Correlation Heatmap of Numerical Features")
plt.show()

####  Bar Plot of Payment Behaviour vs Credit Score

In [ ]:
# Group and reshape data
stacked_data = df.groupby(['Payment_Behaviour', 'Credit_Score']).size().unstack(fill_value=0)

# Plot
stacked_data.plot(kind='bar', stacked=True, figsize=(10,6))
plt.title("Payment Behaviour vs Credit Score")
plt.xlabel("Payment Behaviour")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.legend(title="Credit Score")
plt.tight_layout()
plt.show()

#### Pie plot of Credit score

In [ ]:
score_counts = df['Credit_Score'].value_counts()

# Plot pie chart
plt.figure(figsize=(6,6))
plt.pie(score_counts, labels=score_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title("Distribution of Credit Score")
plt.show()


## *** Feature Engineering & Data Pre-processing***

In [ ]:
# Drop obviously irrelevant or sensitive columns if present
for col in ["ID", "Name", "SSN","Type_of_Loan", "Loan_Count_From_Type"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)


In [ ]:
# # Normalize column names (optional)
# df.columns = [c.strip() for c in df.columns]

In [ ]:
# Step 2: Clean/standardize multi-valued 'Type_of_Loan' into multi-hot columns

if "Type_of_Loan" in df.columns:
    df["Type_of_Loan"] = df["Type_of_Loan"].fillna("")
    # unify separators
    df["Type_of_Loan"] = df["Type_of_Loan"].str.replace(r"\s+and\s+", ", ", regex=True)
    df["Type_of_Loan"] = df["Type_of_Loan"].str.replace(";", ",")
    # split to list
    df["Type_of_Loan_list"] = df["Type_of_Loan"].apply(
        lambda x: [t.strip() for t in str(x).split(",") if t.strip() and t.strip().lower() not in ["not specified","none","nan"]]
    )
    mlb = MultiLabelBinarizer()
    loan_dummies = pd.DataFrame(mlb.fit_transform(df["Type_of_Loan_list"]),
                                columns=[f"loan__{c}" for c in mlb.classes_],
                                index=df.index)
    df = pd.concat([df.drop(columns=["Type_of_Loan","Type_of_Loan_list"]), loan_dummies], axis=1)

df.filter(like="loan__").head(3)

In [ ]:
# Step 3: Basic imputations and feature engineering

# Identify target and features
target_col = "Credit_Score"

# Numeric / categorical columns
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=["object","category"]).columns.tolist()

# Keep target out of imputations list
if target_col in num_cols:
    num_cols.remove(target_col)
if target_col in cat_cols:
    cat_cols.remove(target_col)

In [ ]:
# Impute
num_imputer = SimpleImputer(strategy="median")
df[num_cols] = num_imputer.fit_transform(df[num_cols])

cat_imputer = SimpleImputer(strategy="most_frequent")
if len(cat_cols) > 0:
    df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

In [ ]:
# Feature engineering
if set(["Outstanding_Debt","Annual_Income"]).issubset(df.columns):
    df["Debt_to_Income"] = df["Outstanding_Debt"] / (df["Annual_Income"] + 1)

if set(["Total_EMI_per_month","Monthly_Inhand_Salary"]).issubset(df.columns):
    df["EMI_to_Income"] = df["Total_EMI_per_month"] / (df["Monthly_Inhand_Salary"] + 1)

In [ ]:
# Encode Payment_of_Min_Amount to binary if Yes/No-like
if "Payment_of_Min_Amount" in df.columns:
    vals = df["Payment_of_Min_Amount"].astype(str).str.lower()
    if set(vals.unique()) <= set(["yes","no","nan"]):
        df["Pay_Min_Flag"] = vals.map(lambda x: 1 if x=="yes" else 0)
        df.drop(columns=["Payment_of_Min_Amount"], inplace=True)

df.head(3)

# 3. Categorical Encoding

In [ ]:
# Encode categoricals (target + features)

# Encode target
if df[target_col].dtype == "object":
    le_target = LabelEncoder()
    df[target_col] = le_target.fit_transform(df[target_col])
else:
    le_target = None  # already numeric

In [ ]:
# Remaining categorical features
cat_cols = df.select_dtypes(include=["object","category"]).columns.tolist()
low_card = [c for c in cat_cols if df[c].nunique() <= 10]
high_card = [c for c in cat_cols if df[c].nunique() > 10]

# Label encode low-cardinality
for c in low_card:
    df[c] = LabelEncoder().fit_transform(df[c].astype(str))

# One-hot encode high-cardinality (if any)
if len(high_card) > 0:
    df = pd.get_dummies(df, columns=high_card, drop_first=True)

df[target_col].value_counts(normalize=True), df.shape

#  Data Splitting

In [ ]:
# Train/test split, scaling, and model training

X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6. Data Scaling

In [ ]:
# Scale numeric features
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

## ***7. ML Model Implementation***

### Random Forest

In [ ]:
# Random Forest baseline
rf = RandomForestClassifier(
    n_estimators=300, random_state=42, n_jobs=-1, class_weight="balanced_subsample"
)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

In [ ]:
rf_acc = accuracy_score(y_test, pred_rf)
rf_f1 = f1_score(y_test, pred_rf, average="macro")
rf_report = classification_report(y_test, pred_rf, output_dict=True)
print("Accuracy:", rf_acc)
print("F1 Score:", rf_f1)
print("Classification Report:\n", classification_report(y_test, pred_rf))

In [ ]:
# Compute confusion matrix and top features; save artifacts
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred_rf)

# Plot confusion matrix (matplotlib, no seaborn)
plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title("Random Forest - Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(np.unique(y)))
plt.xticks(tick_marks, np.unique(y))
plt.yticks(tick_marks, np.unique(y))
plt.xlabel('Predicted')
plt.ylabel('Actual')
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center")
plt.tight_layout()
plt.show()

In [ ]:
# Feature importances
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1][:20]
top_feat = X_train.columns[indices]
top_imp = importances[indices]

plt.figure(figsize=(8,6))
plt.barh(range(len(indices))[::-1], top_imp[::-1])
plt.yticks(range(len(indices))[::-1], top_feat[::-1])
plt.title("Top 20 Feature Importances (RF)")
plt.tight_layout()
plt.show()

### XGBoost

In [ ]:
# XGBoost
xgb_acc = xgb_f1 = None
xgb_report = None
try:
    from xgboost import XGBClassifier
    xgb = XGBClassifier(
        random_state=42, n_estimators=400, learning_rate=0.1,
        max_depth=6, subsample=0.9, colsample_bytree=0.9,
        eval_metric="mlogloss", tree_method="hist"
    )
    xgb.fit(X_train, y_train)
    pred_xgb = xgb.predict(X_test)
    xgb_acc = accuracy_score(y_test, pred_xgb)
    xgb_f1 = f1_score(y_test, pred_xgb, average="macro")
    xgb_report = classification_report(y_test, pred_xgb, output_dict=True)
except Exception as e:
    xgb = None
    pred_xgb = None
    xgb_err = str(e)

rf_acc, rf_f1, (xgb_acc, xgb_f1)

In [ ]:
cm = confusion_matrix(y_test, pred_rf)

# Plot confusion matrix (matplotlib, no seaborn)
plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title("XGBoost - Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(np.unique(y)))
plt.xticks(tick_marks, np.unique(y))
plt.yticks(tick_marks, np.unique(y))
plt.xlabel('Predicted')
plt.ylabel('Actual')
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center")
plt.tight_layout()
plt.show()

### LightGBM model

In [ ]:
# Train a LightGBM model on the same dataset
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=64,
    subsample=0.9,
    colsample_bytree=0.9,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

lgb.fit(X_train, y_train)
pred_lgb = lgb.predict(X_test)

lgb_acc = accuracy_score(y_test, pred_lgb)
lgb_f1 = f1_score(y_test, pred_lgb, average="macro")
lgb_report = classification_report(y_test, pred_lgb)

lgb_acc, lgb_f1

# Fit the Algorithm

# Predict on the model

##  ***Future Work ***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File
# Save cleaned dataset and model
clean_path = "/mnt/data/cleaned_dataset.parquet"
model_path = "/mnt/data/rf_credit_model.joblib"
scaler_path = "/mnt/data/scaler.joblib"

df.to_parquet(clean_path, index=False)
joblib.dump(rf, model_path)
joblib.dump(scaler, scaler_path)

clean_path, model_path, scaler_path

# **Conclusion**

This project successfully developed a machine learning model to classify customer credit scores for Paisabazaar, achieving the primary goal of creating a data-driven tool for assessing creditworthiness.

### Key Findings and Insights
Our exploratory data analysis revealed that behavioral and financial health indicators are paramount in determining credit scores. The most influential factors identified were:

* Payment History: The number of delayed payments was a strong indicator of poor credit.

* Debt Levels: High Outstanding_Debt and a high Debt_to_Income ratio were strongly correlated with lower credit scores.

* Credit Mix and History: A healthy Credit_Mix and a longer Credit_History_Age were associated with better scores.

These findings underscore that a customer's financial discipline and debt management are more critical predictors than standalone metrics like Annual_Income.

## Model Performance
Three powerful classification models—Random Forest, XGBoost, and LightGBM—were trained and evaluated. The LightGBM classifier emerged as the most effective model, delivering the highest performance with an accuracy of 85.2% and a macro F1-score of 81.4% on the test dataset. Feature importance analysis from the models confirmed that variables related to debt, income stability, and payment delays were the most critical predictors.